In [19]:
import pandas as pd # type: ignore
pd.set_option('display.max_columns', None)
df_roster = pd.read_csv("2022-2023/22-23-Roster.csv", skiprows= [1]) #deletes unwanted repeated row of attributes
df_game_logs = pd.read_csv("2022-2023/22-23-game-logs.csv")
df_per_game = pd.read_csv("2022-2023/22-23-Per-Game.csv")
df_results = pd.read_csv("2022-2023/22-23-Results.csv")
df_team_n_opp = pd.read_csv("2022-2023/22-23-Team&Opp-Per-Game.csv")

In [25]:
df_team_n_opp
df_team_n_opp = df_team_n_opp.drop(columns= ['Unnamed: 0'])
df_team_n_opp.rename(columns = {'Unnamed: 1' : ''}, inplace = True)
df_team_n_opp

,,G,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Team,33.0,40.5,26.5,59.0,.449,18.9,34.3,.551,7.6,24.7,.308,13.7,20.2,.679,11.1,26.7,37.8,12.4,6.9,5.6,12.9,16.9,74.3
1,Rank,NaN,NaN,113th,111th,167th,123rd,250th,24th,157th,52nd,340th,116th,71st,310th,91st,38th,36th,237th,120th,2nd,127th,165th,116th
2,Opponent,33.0,40.5,24.4,58.6,.416,18.4,40.8,.451,6.0,17.8,.334,12.5,17.7,.705,9.8,24.0,33.8,10.7,6.9,2.8,13.0,18.3,67.2
3,Rank,NaN,NaN,129th,237th,56th,219th,347th,20th,37th,29th,159th,161st,177th,128th,189th,140th,156th,127th,272nd,102nd,226th,299th,88th


In [11]:
#deletes 'Unnamed: 0' column from the dataframes
df_roster = df_roster.drop(df_roster.columns[0], axis = 1)
df_game_logs = df_game_logs.drop(df_game_logs.columns[0], axis = 1)
df_per_game = df_per_game.drop(df_per_game.columns[0], axis = 1)
df_results = df_results.drop(df_results.columns[0], axis = 1)
df_team_n_opp = df_team_n_opp.drop(df_team_n_opp.columns[0], axis = 1)

In [ ]:
#goal: clean up df_roster and make it useable by converting string values into integers as appropriate 
df_rosterc = df_roster.drop(['RSCI Top 100', 'High School', 'Hometown'], axis = 1) #delete RSCI Top 100 col because unneeded 
df_rosterc = df_rosterc.join(df_per_game[['PTS','AST']]) #combines columns from df_per_game as they are the same as some of the summary 
def extract_rebounds(summary):
    if 'Reb' in summary:
        # Split by commas and find the part containing 'Reb'
        parts = summary.split(', ')
        for part in parts:
            if 'Reb' in part:
                return float(part.split(' ')[0])  # Extract the number before 'Reb'
    return 0.0  # Return 0.0 if no 'Reb' is found
df_rosterc['REB'] = df_rosterc['Summary'].apply(extract_rebounds)
df_rosterc



In [ ]:
df_per_game #all values are good as is, potentially look into removing nan values but as it stands, it's alright

In [ ]:
df_resultsc = df_results.drop(['Unnamed: 5'], axis = 1)
df_resultsc.rename(columns = {'Unnamed: 9' : 'Result'}, inplace = True)
df_resultsc['Home Game'] = df_resultsc['Arena'] == 'State Farm Center'
df_resultsc = df_resultsc.drop(columns = 'Arena')
df_resultsc['Streak'] = df_resultsc['Streak'].str.split().str[-1].astype(int)
df_resultsc.insert(5, 'Ranked Match', None)
df_resultsc.insert(5, 'Rank', None)
df_resultsc['Rank'] = df_resultsc['Opponent'].str.extract(r'\((\d+)\)')
df_resultsc['Ranked Match'] = df_resultsc['Rank'].notna()
df_resultsc

In [ ]:
df_game_logs = df_game_logs.drop(columns = ['G', 'Unnamed: 3'])
df_game_logs

In [ ]:
df_merge_glr = pd.merge(df_resultsc, df_game_logs, left_index= True, right_index= True, how = 'inner')
df_merge_glr = df_merge_glr.drop(columns=['W/L', 'Date_x', 'Opp_y', 'Tm_y', 'Opp.1', 'Unnamed: 24'])
date_col = df_merge_glr.pop('Date_y')
df_merge_glr.insert(0, 'Date', date_col)
df_merge_glr.rename(columns = {'Tm_x' : 'PF', 'Opp_x' : 'PA'}, inplace = True)
df_merge_glr.columns =  [f'OPP {col[:-2]}' if '.1' in col else col for col in df_merge_glr.columns]
df_merge_glr


In [27]:
# Export all to csvs
df_rosterc.to_csv(r'C:/Users/lyons/OneDrive/Desktop/DataDiveFa24/Clean Datasets/Clean-22-23-Roster.csv')
df_team_n_opp.to_csv(r'C:/Users/lyons/OneDrive/Desktop/DataDiveFa24/Clean Datasets/Clean-22-23-Team&Opponents.csv')
df_merge_glr.to_csv(r'C:/Users/lyons/OneDrive\Desktop\DataDiveFa24/Clean Datasets/Clean-22-23-Game-log.csv')
